# FIT3182 Big data management and processing - S1 2021 Assignment
***

Name: Wong Kai Lin

Student ID: 30507588

Email: kwon0061@student.monash.edu
***


## Part B- 
## Task 1. Processing Data Stream 
### c. Event Producer 3:

Write a python that loads all the data from hotspot_TERRA_streaming.csv and randomly (with replacement) feeds the data to the stream every 2 seconds. 

TERRA is another satellite from NASA that reports latitude, longitude, confidence and surface temperature of a location. 

You will need to append additional information 
such as producer information to identify the producer and 
created date & time.

In [1]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
from kafka import KafkaConsumer
import datetime as dt
import random
import datetime as dt
from datetime import datetime
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json

In [2]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [11]:
# retrieve current climate date in Kafka

def climate_date():
    
    def connect_kafka_consumer():
        _consumer = None
        try:
             _consumer = KafkaConsumer(topic,
                                       consumer_timeout_ms=10000, # stop iteration if no message after 10 sec
                                       auto_offset_reset='earliest', # comment this if you don't want to consume earliest available message
                                       bootstrap_servers=['localhost:9092'],
                                       api_version=(0, 10))
        except Exception as ex:
            print('Exception while connecting Kafka')
            print(str(ex))
        finally:
            return _consumer
 
    def consume_messages(consumer):
        
        try:
            
            # ATTEMPT TO ONLY READ THE LAST MESSAGE IN KAFKA
            #consumer.poll()
            #consumer.seek_to_end()
            
            #pos = consumer.position()

            for msg in consumer:
                message = msg

            # only keep the latest climate record
            data = str(message.value.decode('utf-8')).split(', ') # decode bytes message from kafka producer

            # convert message in kafka back into JSON
            string_json = ""
            for i in data:
                string_json+=i # combine all the key values pairs into one string_json
                string_json+=',' # append a comma to separate each key

            string_json = string_json[:-1] # remove last , from string_json
            climate_json = json.loads(string_json) # parse string_json into json            

            # acess date value  
            climate_date = climate_json['date']

        except Exception as ex:
            print(str(ex))

        finally:
            return climate_date
    
    if __name__ == '__main__':
        topic = 'climate'
        consumer = connect_kafka_consumer()
        return consume_messages(consumer)
        
#climate_date()        

In [12]:
def publish_message(producer_instance, topic_name, data):
    
    try:    

        data = json.dumps(data) # dictionary data of climate converted to json string             

        value_bytes = bytes(data, encoding='utf-8') # encode json string data to bytes

        key_bytes = bytes("P3", encoding='utf-8') # specific key for each producer

        producer_instance.send(topic_name, value=value_bytes, key=key_bytes)
        
        print('Message published successfully. Data: ' + data)
        
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [13]:
if __name__ == '__main__':
   
    topic = 'hotspot'
    print('Publishing records..')
    producer3 = connect_kafka_producer()
    
    # reading csv
    hotspot_terra = pd.read_csv("hotspot_TERRA_streaming.csv")
    head = list(hotspot_terra.columns) # column names    

    while True:
        row = random.randrange(0, len(hotspot_terra)-1) # retrieving a random row data from hotspot_terra
               
        hotspot_terra_json = {} # json to store the row of climate data
        
        for cols in head:
            hotspot = hotspot_terra.loc[row,cols] #value of each column of the data row            
            hotspot_terra_json[cols] = hotspot # append column name and data value into json

        # convert numpy.int64 column values to int for JSON
        hotspot_terra_json['confidence'] = int(hotspot_terra_json['confidence'])
        hotspot_terra_json['surface_temperature_celcius'] = int(hotspot_terra_json['surface_temperature_celcius']) 
        
        # accessing the current (latest) climate date to be the hotspot date
        # ISSUE: climate_date function takes too long to run as it loops to the last consumer message
        
        #date = climate_date()
         
        # date time now
        now = datetime.now()
        date = now.strftime("%Y-%m-%d")
        current_time = now.strftime("%H:%M:%S")
        
        data = {'producer': 2, 'climate': hotspot_terra_json, 'date': date, 'time': current_time}

        publish_message(producer3, topic, data)
        
        sleep(2) # feed data to spark streaming every 2 seconds

Publishing records..
Message published successfully. Data: {"time": "21:00:46", "climate": {"longitude": 142.8819, "latitude": -37.6073, "surface_temperature_celcius": 57, "confidence": 83}, "producer": 2, "date": "2021-05-23"}
Message published successfully. Data: {"time": "21:00:48", "climate": {"longitude": 145.6489, "latitude": -35.9435, "surface_temperature_celcius": 51, "confidence": 78}, "producer": 2, "date": "2021-05-23"}
Message published successfully. Data: {"time": "21:00:50", "climate": {"longitude": 142.5943, "latitude": -36.5642, "surface_temperature_celcius": 38, "confidence": 51}, "producer": 2, "date": "2021-05-23"}
Message published successfully. Data: {"time": "21:00:52", "climate": {"longitude": 141.8857, "latitude": -37.094, "surface_temperature_celcius": 57, "confidence": 83}, "producer": 2, "date": "2021-05-23"}
Message published successfully. Data: {"time": "21:00:54", "climate": {"longitude": 147.2232, "latitude": -37.8323, "surface_temperature_celcius": 68, "

KeyboardInterrupt: 